In [ ]:
!pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=06118e6bdab2bd2a4b895bc446c254ae280752cfa392221c10ad965bcd77c6ed
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [1]:
import pandas as pd
import re
import nltk
import numpy as np
from autocorrect import Speller
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

drive_url = "/content/drive/MyDrive/LSTM-GRU-data/"

Mounted at /content/drive


In [2]:
dataset_target = "Dataset/"
img_targer = "Img/"
model_checkpoint_target = "Model_checkpoint/"
preprocessed_target = "Preprocessed/"
pretrained_target = "Pretrained/"

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
spell = Speller(lang='en')
stop_words = set(stopwords.words('english'))

removal_positive = ['but', "aren't", "couldn't", "didn't", "doesn't", "don't", "hadn't", "hasn't", "haven't", "isn't",
                    "mightn't", "mustn't", "needn't", "shan't", "shouldn't", "wasn't", "won't", "wouldn't"]
stop_words.difference_update(removal_positive)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'by',
 'can',
 'couldn',
 'd',
 'did',
 'didn',
 'do',
 'does',
 'doesn',
 'doing',
 'don',
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 'has',
 'hasn',
 'have',
 'haven',
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 'more',
 'most',
 'mustn',
 'my',
 'myself',
 'needn',
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's',
 'same',
 'shan',
 'she',
 "she's",
 'should',
 "should've",
 'shouldn',
 'so',
 'some',
 'such',
 't',
 'than',
 'that',
 "that'll",
 '

1. Import data
2. Define column to be cleaned
3. clean each row of that column
4. make it as column after cleaned -> to csv
5. from cleaned csv to column to augment
6. copy a subset to be augmented
7. augment that subset -> to csv

# 1 & 2

In [ ]:
data = pd.read_csv(drive_url+dataset_target+"text_emotion.csv")
# data = data.head(5)
columns = ["id", "text", "label"]
data.columns = columns

# truncate class: surprise
data = data[data["label"] != "surprise"]

# truncate duplicate by A
data = data.drop_duplicates(subset=['text'])

# convert label by class: {pos:1, neg:0}
data['label'].replace({'positive': 1, 'negative': 0}, inplace=True)
data["label"].value_counts()

<ipython-input-17-8023912c660c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'].replace({'positive': 1, 'negative': 0}, inplace=True)


0    218838
1    165154
Name: label, dtype: int64

# 3

In [ ]:
def reduce_lengthening(text: str) -> str:
    """
    Reduce consecutive character lengthening in the input text.

    This function identifies consecutive character lengthening (e.g., "sooo" or "coooool") in the text and replaces them with double characters (e.g., "soo" or "cool").

    Parameters:
        text (str): The input text containing potential character lengthening.

    Returns:
        str: The text with consecutive character lengthening reduced.

    Example:
        >>> reduced_text = reduce_lengthening("sooo good!")
        >>> print(reduced_text)
        "soo good!"
    """
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)


def text_preprocess(doc: str) -> str:
    """
    Perform comprehensive text preprocessing on the input document.

    This function performs several text preprocessing steps, including lowercase conversion, removal of hashtags, mentions, links, numbers, and more.
    It also tokenizes the text, reduces word lengthening, corrects spelling, lemmatizes words, and removes short words.

    Parameters:
        doc (str): The input document to be preprocessed.

    Returns:
        str: The preprocessed text.

    Example:
        >>> preprocessed_text = text_preprocess("I love this product! It's amazing!!!")
        >>> print(preprocessed_text)
        "love product amazing"
    """
    #Lowercasing all the letters
    temp = doc.lower()
    #Removing hashtags and mentions
    temp = re.sub("@[A-Za-z0-9_]+", "", temp)
    temp = re.sub("#[A-Za-z0-9_]+", "", temp)
    #Removing links
    temp = re.sub(r"http\S+", "", temp)
    temp = re.sub(r"www.\S+", "", temp)
    #removing numbers
    temp = re.sub("[0-9]", "", temp)
    #Removing '
    temp = re.sub("'", " ", temp)

    #Tokenization
    temp = word_tokenize(temp)
    #Fixing Word Lengthening
    temp = [reduce_lengthening(w) for w in temp]
    #spell corrector
    temp = [spell(w) for w in temp]
    #stem
    temp = [lemmatizer.lemmatize(w) for w in temp]
    #Removing short words
    temp = [w for w in temp if len(w) > 2]

    temp = " ".join(w for w in temp)

    return temp


examples_test_cleaning_text = "I don't feel good today"

text_preprocess(examples_test_cleaning_text)

'don feel good today'

In [ ]:
cleaned_text = [text_preprocess(sent) for sent in data['text']]

In [ ]:
np.save(drive_url+preprocessed_target+"cleaned_text.npy",cleaned_text)

In [ ]:
data['text'] = cleaned_text

In [ ]:
len(cleaned_text)

383992

# 4

In [ ]:
data.to_csv(drive_url+dataset_target+"cleaned_text_emotion.csv")

# 5

In [ ]:
positive_data = data[data['label']==1]
negative_data = data[data['label']==0]
positive_data_sample = positive_data.sample(frac=0.33,random_state=17)
positive_data_sample['label'].value_counts()

1    54501
Name: label, dtype: int64

In [ ]:
positive_data['label'].value_counts()

1    165154
Name: label, dtype: int64

In [ ]:
np.set_printoptions(threshold=np.inf)
positive_data['text'].head(5)

2    ive probably mentioned this before but really ...
4    believe that much more sensitive other people ...
5    find myself frustrated with christian because ...
6    one those people who feel like going the gym o...
7    feel especially pleased about this this been l...
Name: text, dtype: object

In [ ]:
negative_data['label'].value_counts()

0    218838
Name: label, dtype: int64

In [ ]:
negative_data['text'].head(5)

0     feel awful about too because job get him posit...
1                                      alone feel awful
3                       feeling little low few day back
9               feel enraged but helpless the same time
10                          said feeling bit rebellious
Name: text, dtype: object